In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importing all required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
spark = SparkSession.builder.appName('Test').getOrCreate()

In [3]:
# Reading the Dataframe from the csv file.
df = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/Airline_data.csv")  
df.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

## Show a sample of 5 records from dataset

In [4]:
df.show(5)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1552|           UA|      183

## Read the data with data types.

In [5]:
# Reading datatypes for all the columns
for col in df.dtypes:
    print(col[0]+" :- "+col[1])

Year :- int
Month :- int
DayofMonth :- int
DayOfWeek :- int
DepTime :- int
CRSDepTime :- int
ArrTime :- int
CRSArrTime :- int
UniqueCarrier :- string
FlightNum :- int
TailNum :- string
ActualElapsedTime :- int
CRSElapsedTime :- int
AirTime :- string
ArrDelay :- int
DepDelay :- int
Origin :- string
Dest :- string
Distance :- int
TaxiIn :- string
TaxiOut :- string
Cancelled :- int
CancellationCode :- string
Diverted :- int
CarrierDelay :- string
WeatherDelay :- string
NASDelay :- string
SecurityDelay :- string
LateAircraftDelay :- string


## Make a new column MonthStr, Which has months in form of 01, 02, 03, ..., 12

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def somefunc(value):
  if   value == 1: 
      return '01'
  elif value == 2:
      return '02'
  elif value == 3:
      return '03'
  elif value == 4:
      return '04'
  elif value == 5:
      return '05'
  elif value == 6:
      return '06'
  elif value == 7:
      return '07'
  elif value == 8:
      return '08'
  elif value == 9:
      return '09'
  elif value == 10:
      return '10'
  elif value == 11:
      return '11'
  elif value == 12:
      return '12'
      

#convert to a UDF Function by passing in the function and return type of function
udfsomefunc = F.udf(somefunc, StringType())
df2 = df.withColumn("MonthStr", udfsomefunc("Month"))
df2.show()

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|MonthStr|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+--------+
|1989|    1|        23|        1|   1419|      1230|   1742|      1

## Find the # of flights each airline made

In [11]:
flights = df.groupBy("FlightNum").count()
flights.show(flights.count(),truncate=False)

+---------+-----+
|FlightNum|count|
+---------+-----+
|193      |24   |
|183      |9    |
|192      |4    |
|190      |31   |
|185      |88   |
|191      |30   |
|188      |59   |
|184      |121  |
|186      |29   |
|187      |31   |
+---------+-----+



## Find the mean departure delay per origination airport.

In [9]:
dep_delay_origin = df.groupBy("Origin").mean("DepDelay")
dep_delay_origin.withColumnRenamed("avg(DepDelay)", "mean(DepDelay)").show(dep_delay_origin.count(),truncate=False)

+------+-------------------+
|Origin|mean(DepDelay)     |
+------+-------------------+
|LIH   |-3.7666666666666666|
|HNL   |3.217741935483871  |
|EWR   |4.958333333333333  |
|DEN   |27.6               |
|IAD   |8.9                |
|SFO   |19.646153846153847 |
|PHL   |16.137931034482758 |
|OGG   |6.0                |
+------+-------------------+



## What is the average departure delay from each airport?

In [10]:
dep_delay_origin = df.groupBy("Dest").mean("DepDelay")
dep_delay_origin.withColumnRenamed("Dest", "Airport").show(dep_delay_origin.count(),truncate=False)

+-------+------------------+
|Airport|avg(DepDelay)     |
+-------+------------------+
|LIH    |6.4               |
|HNL    |14.252525252525253|
|DEN    |16.774193548387096|
|IAD    |27.6              |
|PDX    |24.333333333333332|
|SFO    |7.3931623931623935|
|LAX    |6.956896551724138 |
+-------+------------------+

